In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./1.make.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"jsn":{"tab":"catalog","make":"' + input_.loc[i, 'Make'] + '","nodetype":"make"}}'

                payload = {'func': 'navnode_fetch',
                           'payload': jsn,
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_year = [int(year.strip()) for year in html.xpath('//td[@class="nlabel"]/a/text()')]
                
                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                    'No.': int(input_.loc[i, 'No.']),
                                    'Make': input_.loc[i, 'Make'],
                                    'Year': list_year,
                                    'Url': list_url})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        else:
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Make': input_.loc[i, 'Make'],
                                     'Url': input_.loc[i, 'Url']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Make'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Year'], ascending=[True, False])
output_correct.to_excel('./2.year-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True])
    output_error.to_excel('./year_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：268

AMPHICAR  <->  [ok] - 剩余数量：248 - 当前时间：14:50:00
ASUNA  <->  [ok] - 剩余数量：247 - 当前时间：14:50:00
AMERICAN BANTAM  <->  [ok] - 剩余数量：246 - 当前时间：14:50:00
ABARTH  <->  [ok] - 剩余数量：245 - 当前时间：14:50:00
ALLSTATE  <->  [ok] - 剩余数量：244 - 当前时间：14:50:00
ACURA  <->  [ok] - 剩余数量：243 - 当前时间：14:50:00
AC  <->  [ok] - 剩余数量：242 - 当前时间：14:50:00
ARMSTRONG-SIDDELEY  <->  [ok] - 剩余数量：241 - 当前时间：14:50:00
ARNOLT-MG  <->  [ok] - 剩余数量：240 - 当前时间：14:50:00
ALFA ROMEO  <->  [ok] - 剩余数量：239 - 当前时间：14:50:00
ALLARD  <->  [ok] - 剩余数量：238 - 当前时间：14:50:00
ALPINE  <->  [ok] - 剩余数量：237 - 当前时间：14:50:00
ARNOLT-BRISTOL  <->  [ok] - 剩余数量：236 - 当前时间：14:50:00
ALVIS  <->  [ok] - 剩余数量：235 - 当前时间：14:50:00
AM GENERAL  <->  [ok] - 剩余数量：234 - 当前时间：14:50:00
AMERICAN MOTORS  <->  [ok] - 剩余数量：233 - 当前时间：14:50:00
AUBURN  <->  [ok] - 剩余数量：232 - 当前时间：14:50:00
AMERICAN AUSTIN  <->  [ok] - 剩余数量：231 - 当前时间：14:50:00
AUSTIN-HEALEY  <->  [ok] - 剩余数量：230 - 当前时间：14:50:00
AUDI  <->  [ok] - 剩余数量：229 - 当前时间：14:50:00
APOLLO  <->  [ok] - 剩余数量：228 - 

OAKLAND  <->  [ok] - 剩余数量：67 - 当前时间：14:50:07
OPEL  <->  [ok] - 剩余数量：66 - 当前时间：14:50:07
OMEGA  <->  [ok] - 剩余数量：65 - 当前时间：14:50:07
OLDSMOBILE  <->  [ok] - 剩余数量：64 - 当前时间：14:50:07
PAIGE  <->  [ok] - 剩余数量：63 - 当前时间：14:50:07
PACKARD  <->  [ok] - 剩余数量：62 - 当前时间：14:50:07
OSCA  <->  [ok] - 剩余数量：61 - 当前时间：14:50:07
PANHARD  <->  [ok] - 剩余数量：60 - 当前时间：14:50:07
PETERBILT  <->  [ok] - 剩余数量：59 - 当前时间：14:50:08
PANOZ  <->  [ok] - 剩余数量：58 - 当前时间：14:50:08
PASSPORT  <->  [ok] - 剩余数量：57 - 当前时间：14:50:08
PORSCHE  <->  [ok] - 剩余数量：56 - 当前时间：14:50:08
PEERLESS  <->  [ok] - 剩余数量：55 - 当前时间：14:50:08
PLYMOUTH  <->  [ok] - 剩余数量：54 - 当前时间：14:50:08
PANTHER  <->  [ok] - 剩余数量：53 - 当前时间：14:50:08
PEGASO  <->  [ok] - 剩余数量：52 - 当前时间：14:50:08
PEUGEOT  <->  [ok] - 剩余数量：51 - 当前时间：14:50:08
PIERCE-ARROW  <->  [ok] - 剩余数量：50 - 当前时间：14:50:08
POLESTAR  <->  [ok] - 剩余数量：49 - 当前时间：14:50:08
PONTIAC  <->  [ok] - 剩余数量：48 - 当前时间：14:50:08
QVALE  <->  [ok] - 剩余数量：47 - 当前时间：14:50:08
RELIANT  <->  [ok] - 剩余数量：46 - 当前时间：14:50:08
RILEY  <-> 